In [1]:
# print current date
import datetime
print(f'Updated: {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

Updated: 2024-05-06 11:36:16


In [1]:
import xarray as xr
import os
import glob
import dask.distributed as dd
import dask
import dask.array as da
import time
import sys
import calendar
from datetime import datetime, timedelta
import numpy as np
import wrf
from itertools import product

import wrf
from wrf import interplevel

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['text.usetex'] = False

import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cartopy
from cartopy import crs
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors

from windrose import WindroseAxes

from scipy.stats import weibull_min

import seaborn as sns
import seaborn.objects as so

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import AutoMinorLocator

from scipy.stats import linregress


from meteostat import Stations
from datetime import datetime
from meteostat import Hourly

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project/WRFV4.4/FLLJ/'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

from data_processing.libraries import *

In [3]:
files = sorted(glob.glob(f'{root_dir}/Elia_wind_power/*.xls'))
dfs = []
for file in files:
    df = pd.read_excel(file,skiprows=2,header=1,index_col=0)
    df.index = pd.to_datetime(df.index,format='%d/%m/%Y %H:%M')
    dfs.append(df)
df = pd.concat(dfs)
df.to_csv(f'{root_dir}/Elia_wind_power/Elia_wind_power.csv')

In [4]:
df

,Week-ahead Forecast [MW],Day-ahead forecast (11h00) [MW],Most recent forecast [MW],Measured & upscaled [MW],Monitored Capacity [MW],Active Decremental Bids [yes/no]
DateTime,,,,,,
2016-01-01 00:00:00,379.45,581.65,541.68,459.57,712.2,No
2016-01-01 00:15:00,377.23,570.27,523.55,419.74,712.2,No
2016-01-01 00:30:00,375.01,558.88,505.42,424.54,712.2,No
2016-01-01 00:45:00,372.79,547.50,487.29,412.59,712.2,No
2016-01-01 01:00:00,370.93,529.31,466.81,402.87,712.2,No
...,...,...,...,...,...,...
2017-01-31 22:45:00,455.79,300.44,225.03,224.20,712.2,No
2017-01-31 23:00:00,453.68,303.85,225.52,211.62,712.2,No
2017-01-31 23:15:00,451.44,307.23,226.02,210.96,712.2,No
